# Distribution of the data related by the following paper: 

### Investigating the effect of grain structure on compressive response of open-cell metal foam using high-fidelity crystal-plasticity modeling 

__Authors:__  _Dongfang Zhao, Kristoffer E. Matheson, Brian R. Phung, Steve Petruzza, Michael W. Czabaj, Ashley D. Spear_ 

__Published in:__  Materials Science and Engineering, Volume 812, 2021, 140847, ISSN 0921-5093

https://doi.org/10.1016/j.msea.2021.140847

https://www.sciencedirect.com/science/article/pii/S0921509321001167

In [1]:
def InstallPrerequisites():

    # Pre-requirements: install all the python libraries needed to run this exemple.
    # You may need to restart the kernel if the process catches a missing library to be installed. 
    import os,sys

    # specific prerequisite (this may need to be completed)
    !"{sys.executable}" -m pip  install --upgrade OpenVisus
    !"{sys.executable}" -m OpenVisus configure 

    # general prerequisites
    !"{sys.executable}" -m pip install pillow
    !"{sys.executable}" -m pip install urllib3
    !"{sys.executable}" -m pip install matplotlib
    !"{sys.executable}" -m pip install mpl_interactions
    !"{sys.executable}" -m pip install ipywidgets
    !"{sys.executable}" -m pip install ipympl 
    !"{sys.executable}" -m pip install ipycanvas
    !"{sys.executable}" -m pip install asyncio
    !"{sys.executable}" -m jupyter nbextension enable --py widgetsnbextension

    print("WARNING: you probably need to restart Jupyter")
    
if False:
    InstallPrerequisites()

In [2]:
import os,sys,io,random
from PIL import Image
from urllib.request import urlopen
import matplotlib 
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display
from PIL import Image
import numpy as np

sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
from OpenVisus import *
print(os.getcwd() )

Starting OpenVisus C:\Users\Vlaerio's PC\AppData\Roaming\Python\Python37\site-packages\OpenVisus\__init__.py 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)] sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0) ...
D:\visus-cache


In [3]:
def Assert(cond):
    if not cond:
        raise Exception("Assert failed")

class CachedDataset(PyDataset):
    
    # constructor
    def __init__(self, args):
        self.local_filename=os.path.abspath(args["local"]).replace("\\","/")
        self.remote_url=args["url"]
        self.remote_access_type = args["access"]
        self.description=args["description"]
        
        #print("local_filename"    ,self.local_filename)
        #print("remote_url"        ,self.remote_url)
        #print("remote_access_type",self.remote_access_type)
        #print("description",       self.description)
        
        super().__init__(LoadDatasetCpp(self.remote_url))
        
        self.num_blocks = len(self.getFields()) * self.getTotalNumberOfBlocks() * len(self.getTimesteps().asVector())
        self.num_blocks_cached = 0

        self.stop_thread=False
        self.thread=None
        
        self.progress=None
        self.progress_display=None

        #print("Database size",self.getWidth(),self.getHeight(),self.getDepth())
        #print("Fields:",self.getFields()) 
        #print("Loaded cached dataset")
        
    # __del__
    def __del__(self):
        self.stopCaching()   
        
    # createAccess
    def createAccess(self, ):
        
        access_config="""
            <access type='multiplex'>
                    <access type='disk' chmod='rw' url='file://{}' />
                    <access type='{}' url='{}' chmod="r" /> 
            </access>  
        """.format(
            self.local_filename.replace("&","&amp;"),
            self.remote_access_type,
            self.remote_url.replace("&","&amp;")) 
        
        # print("Creating access",access_config)

        access= self.createAccessForBlockQuery(StringTree.fromString(access_config))

        # at this point the cache is enabled with the new local idx file
        Assert(os.path.isfile(self.local_filename))

        return access   

    # startCaching
    def startCaching(self, background=True):
        
        if background:
            self.thread = threading.Thread(target=self.startCaching, args=(False,))
            self.stop_thread=False
            self.thread.start()        
            return 

        #print("start caching","...")
        
        access=self.createAccess()

        access.beginRead()
        
        for field in self.getFields():
            for blockid in range(self.getTotalNumberOfBlocks()): 
                for time in self.getTimesteps().asVector():
                    # print("Copying block","time",time,"field",field,"blockid",blockid,"...")
                    buffer =  self.readBlock(blockid, field=field, time=time, access=access)
                    
                     # to debug missing blocks
                    if  False and buffer is None :
                        read_block = db.createBlockQuery(blockid, ord('r'))
                        msg="# {} {} \n".format(blockid,read_block.getLogicBox().toString())
                        os.write(1, bytes(msg,'utf-8'))                   
                    
                    self.num_blocks_cached += 1
                    self.updateProgress()
                    if self.stop_thread:
                        # print("thread stopped")
                        access.endRead()
                        return
                        
        access.endRead()
        self.thread=None
        #print("caching finished done")
        
    # stopCaching
    def stopCaching(self):
        #print("stopping caching...")
        self.stop_thread=True
        if self.thread:
            self.thread.join()
            self.thread=None
    # getWidth
    def getWidth(self):
        p2=self.getLogicBox()[1]
        return p2[0]    
        
    # getHeight
    def getHeight(self):
        p2=self.getLogicBox()[1]
        return p2[1]   
        
    # getDepth
    def getDepth(self):
        p2=self.getLogicBox()[1]
        return p2[2]  
        
    # readSlice
    def readSlice(self,dir=0, slice=0,quality=-3, time=0, access=None):
        
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W] if dir!=0 else [slice,slice+1]
        y=[0,H] if dir!=1 else [slice,slice+1]
        z=[0,D] if dir!=2 else [slice,slice+1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        
        width,height=[value for value in ret.shape if value>1]
        return ret.reshape([width,height])
        
    # setProgress
    def setProgress(self,progress, progress_display):
        self.progress=progress
        self.progress_display=progress_display   
        self.progress.min=0
        self.progress.max =self.num_blocks       

    # updateProgress
    def updateProgress(self):
                    
        if self.progress:
            self.progress.value = self.num_blocks_cached

        if self.progress_display:
            self.progress_display.value = (
                "Caching progress %.2f%% (%d/%d)" % (
                    100 * self.num_blocks_cached/self.num_blocks, 
                    self.num_blocks_cached,
                    self.num_blocks))                    

print("Utilities defined")

Utilities defined


Now you can run a background process that slowly copy blocks from remote location

In [4]:
local_cache="./visus-cache/foam/visus.idx"

sources = [
    {
        "url":"https://mghp.osn.xsede.org/vpascuccibucket1/visus-server-foam/visus.idx?compression=zip&layout=hzorder",
        "access":"CloudStorageAccess",
        "local": local_cache,
        "description":'Open Storage Network (OSN) Pod'
    },
    {
        "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=foam&compression=zip&layout=hzorder",
        "access":"network",
        "local": local_cache,
        "description":'University of Utah Campus Server'
    },
    {
        "url" : "https://s3.us-west-1.wasabisys.com/visus-server-foam/visus.idx?compression=zip&layout=hzorder",
        "access":"CloudStorageAccess",
        "local": local_cache,
        "description": 'Wasabi Commercial Cloud Storage'
    },
    # special random==take any of the above
    {
        "url":"random",
        "access":"random",
        "local":"random",
        "description":"random"
    }
]

def PickSource(index):
    N=len(sources)
    if index==N-1:
        index=random.randint(0,N-2)
    return sources[index]

In [5]:
# if running this code in a Jupter notbeook or JupyterLab
# NOTE I need this cell to be one line only with only this op
%matplotlib notebook 

In [6]:
# function to plot the image data with matplotlib
def ShowData(data, cmap=None, plot=None,width = 6):
    ratio=float(data.shape[1])/data.shape[0]
    fig = plt.figure(figsize = (width,width*ratio))
    plot = plt.imshow(data, origin='lower', cmap=cmap)
    return plot

db=CachedDataset(sources[0])
access=db.createAccess()
first_query=db.readSlice(dir=2, slice=512, access=access, time=0, quality=-3)
#ShowData(first_query)

In [7]:
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

import asyncio
from time import time

def throttle(wait):
    """ Decorator that prevents a function from being called
        more than once every wait period. """
    def decorator(fn):
        time_of_last_call = 0
        scheduled, timer = False, None
        new_args, new_kwargs = None, None
        def throttled(*args, **kwargs):
            nonlocal new_args, new_kwargs, time_of_last_call, scheduled, timer
            def call_it():
                nonlocal new_args, new_kwargs, time_of_last_call, scheduled, timer
                time_of_last_call = time()
                fn(*new_args, **new_kwargs)
                scheduled = False
            time_since_last_call = time() - time_of_last_call
            new_args, new_kwargs = args, kwargs
            if not scheduled:
                scheduled = True
                new_wait = max(0, wait - time_since_last_call)
                timer = Timer(new_wait, call_it)
                timer.start()
        return throttled
    return decorator

In [14]:
class Slices:

    # constructor
    def __init__(self, sources):
        self.sources=sources
        self.db=None
        self.access=None
        self.createGui()
        self.setSource(0)
        self.setDirection(2)
        
    # createGui
    def createGui(self):    
        layout_width = '60%'
        style = {'description_width': 'initial'}

        global first_query
        self.image=widgets.Image(value=self.convertToImage(first_query),format='png',layout=Layout(width=layout_width))
        
        self.source = widgets.Dropdown(
            options=[(source['description'],I) for I,source in enumerate(self.sources)], 
            value=0,
            description='Data source:',
            style=style,
            layout=Layout(width=layout_width))
        self.source.observe(lambda __unused__ : self.setSource(self.source.value))

        self.time  =widgets.IntSlider(
            value=0,
            min=0,
            max=3,
            step=1,
            description="time (0-3)",
            layout=Layout(width=layout_width))
        self.time.observe(lambda __unused__ : self.refreshSlice())

        self.direction = widgets.Dropdown(
            options=[('X', 0), ('Y', 1), ('Z', 2)], 
            value=0,
            description='Slice orthogonal to axis:',
            style=style,
            layout=Layout(width=layout_width))
        self.direction.observe(lambda __unused__ : self.setDirection(self.direction.value))

        self.slice=widgets.IntSlider(
            value=512,
            min=0,
            max=1024,
            step=1,
            description="slice (0-0)",
            layout=Layout(width=layout_width))
        self.slice.observe(lambda __unused__ : self.refreshSliceDelayed())

        self.resolution =widgets.IntSlider(
            value=-2,min=-5,max=0,step=1,
            description="Resolution (coarse=-5,full=0)",
            style=style,
            layout=Layout(width=layout_width))
        self.resolution.observe(lambda __unused__ : self.refreshSlice())

        self.info = widgets.Label("________________________________________________________")
        
        self.progress = IntProgress(min=0, max=0,layout=Layout(width='70%'))
        self.progress_display = Label("Caching progress" +" "*24)
        
        display(self.image)
        display(self.source)
        display(self.time)
        display(self.direction)
        display(self.slice)
        display(self.resolution)
        display(self.info)
        display(HBox([self.progress_display, self.progress]))        
        
    # setSource
    def setSource(self,index,cache_data=True): 
        
        # remove previous source
        if self.db:
            self.db.stopCaching()
            del self.db
            self.db=None
            self.access=None
        
        self.db=CachedDataset(PickSource(index))
        self.access=self.db.createAccess()
        
        if cache_data:
            self.db.startCaching() 
            self.db.setProgress(self.progress,self.progress_display)
        
        self.refreshSlice()   
        
    # setDirection
    def setDirection(self,value):
        self.direction.value=value
        self.slice.max = self.db.getLogicBox()[1][value]-1        
        self.slice.value = self.slice.max//2
        self.slice.description = "slice (0-{})".format(self.slice.max)   
        self.refreshSlice()

    # refreshSlice
    def refreshSlice(self):
        # euristic to map resolution to slice/quality
        resolution = self.resolution.value
        quality = resolution*3
        size_denominator = int(2**(resolution*-1))
        slice = self.slice.value //size_denominator
        slice = slice *size_denominator

        self.info.value = "Time={}, Direction={}, Slice={}  Resolution={} {} , Source={}".format(
            self.time.value,
            ['X', 'Y', 'Z'][self.direction.value],
            slice,
            resolution,
            ["(coarsest)","(coarser) ","(coarse)  ","(medium)  ","(fine)    "," (full)   "] [5+resolution],
            self.source.options[self.source.value][0]) 
    
        data = self.db.readSlice(dir=self.direction.value, slice=slice,quality=quality, time=self.time.value, access=self.access)
        if data is None:
            return
        
        self.image.value = self.convertToImage(data)

    # refreshSlice
    refresh_count = 0
    refresh_frequency = 10
    def refreshSliceDelayed(self):
        self.refresh_count = self.refresh_count+1
        if self.refresh_count == self.refresh_frequency:
            self.refresh_count = 0
            self.refreshSlice()
        else:
            return
        
        
    # convertToImage
    min=max=None
#    @debounce(0.2)
    def convertToImage(self, data):
        buffer = io.BytesIO()
        if self.min == None or self.min > np.amin(data):
            self.min = np.amin(data)
        if self.max == None or self.max < np.amax(data):
            self.max = np.amax(data)
        data = (data - self.min)/  (self.max-self.min)
        image = np.uint8(matplotlib.cm.viridis(data)*255)
        Image.fromarray(image).save(buffer, format='png')
        return buffer.getvalue()        
        
        
slices=Slices(sources)

"Interactive slicing of dataset retrieved from the cloud and cached locally in: "+local_cache

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x10\x00\x00\x02\x00\x08\x06\x00\x00\x00\xd3\xd6U…

Dropdown(description='Data source:', layout=Layout(width='60%'), options=(('Open Storage Network (OSN) Pod', 0…

IntSlider(value=0, description='time (0-3)', layout=Layout(width='60%'), max=3)

Dropdown(description='Slice orthogonal to axis:', layout=Layout(width='60%'), options=(('X', 0), ('Y', 1), ('Z…

IntSlider(value=512, description='slice (0-0)', layout=Layout(width='60%'), max=1024)

IntSlider(value=-2, description='Resolution (coarse=-5,full=0)', layout=Layout(width='60%'), max=0, min=-5, st…

Label(value='________________________________________________________')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed